Made by Pieter de Jong

In [ ]:
import pandas as pd
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
import re
from difflib import ndiff, get_close_matches
import textdistance
import string
import json
import io
import os
import panel as pn
pn.extension()

In [ ]:
def startupCheck():
    if os.path.isfile("claim_matching.json") and os.access("claim_matching.json", os.R_OK):
        # checks if file exists
        print ("File found")
    else:
        print ("Creating file")
        with io.open(os.path.join("", 'claim_matching.json'), 'w') as db_file:
            db_file.write(json.dumps({}))

In [ ]:
startupCheck()

In [ ]:
filename = "C:\\Users\\piete\\AppData\\Roaming\\MobaXterm\\slash\\RemoteFiles\\2099638_2_1\\2020-2022_BAK Cakes&Sweet_Goods_WE1 (8300).xls"

In [ ]:
#take Event Date, Product Name and Claims/Features from all sheets in the excel file
df = pd.concat(pd.read_excel(filename, usecols=["Event Date", "Product Name", "Claims/Features"], sheet_name=None), ignore_index=True)

In [ ]:
#path = "/commons/dsls/fine_bakery/Data/""
#files = os.listdir(path)
#files_xls = [f for f in files if f[-3:] == 'xls']
#df = pd.DataFrame()
#for f in files_xls:
#    data = pd.read_excel(f, usecols=["Event Date", "Product Name", "Claims/Features"], sheet_name=None), ignore_index=True
#    df = df.append(data)

In [ ]:
#drops rows without claims
df = df.dropna(subset=["Claims/Features"])

In [ ]:
#Make all claims lowercase
df["Claims/Features"] = df["Claims/Features"].str.lower()
#Cleaning by renaming and removing some characters
df["Claims/Features"] = df["Claims/Features"].str.replace(",", ".").str.replace("\n", " ").str.replace("\'s", "")

In [ ]:
#remove last dot in each claims/features. then turn the claims/features into a list of claims by splitting the content by the dots
df["claims_proccesed"] = df["Claims/Features"].str.rstrip(".").str.split("\. ")

In [ ]:
df

In [ ]:
#function that returns claims containing provided pattern
def find_pattern(pattern, string):
    return bool(re.search(pattern, string))

In [ ]:
#removing claims that contain patterns marking them non claims
all_prod_claims = []
pattern = ": \d|kcal|kj|\dg|\d g|.org"
for claims in df["claims_proccesed"]:
    claims_no_ingredients = []
    for claim in claims:
        claim = claim.lstrip()
        if not find_pattern(pattern, claim):
            claims_no_ingredients.append(claim)
    all_prod_claims.append(claims_no_ingredients)
df["claims_proccesed"] = all_prod_claims

pattern = ": \d|kcal|kj|\dg|\d g|.org"
claims_no_ingredients = []
for claim in all_claims:
    if not find_pattern(pattern, claim):
        claims_no_ingredients.append(claim)

In [ ]:
#pd.options.display.max_colwidth = 1000

In [ ]:
#splitting sentences where a space was missing. including when next sentence starts with a number.
#does not split abreviations like h.u.v and ignores no.1

pattern = "\D\.\D"
pattern2 = "\D\.\D\."
pattern3 = "\D\.\d"
pattern4 = "no.1"
claims_cleaned = []
for claims in df["claims_proccesed"]:
    temp_claims = claims
    for claim in claims:
        if find_pattern(pattern, claim) and not find_pattern(pattern2, claim):
            temp_claims.remove(claim)
            temp_claims.append(claim.split(".")[0])
            temp_claims.append(claim.split(".")[1])
        

        if find_pattern(pattern3, claim) and not find_pattern(pattern4, claim):
            temp_claims.remove(claim)
            temp_claims.append(claim.split(".")[0])
            temp_claims.append(claim.split(".")[1])
    claims_cleaned.append(temp_claims)
df["claims_proccesed"] = claims_cleaned

        

In [ ]:
#create list of all unique claims and a list of all claims
all_cleaned_unique_claims = []
all_cleaned_claims = []
for claims in df["claims_proccesed"]:
    for claim in claims:
        all_cleaned_claims.append(claim)
        if claim not in all_cleaned_unique_claims:
            all_cleaned_unique_claims.append(claim)

In [ ]:
Claim_ammount = Counter(all_cleaned_claims)
Claim_ammount.most_common()

In [ ]:
#find all claims containing pattern and adding these claims to claim_dict with pattern as key if their tickbox is selected
claim_dict = {}
pattern = "vegetarian"
def get_matches(pattern):
    #pattern = "vegan"
    pattern_match = []
    for claim in all_cleaned_unique_claims:
        if find_pattern(pattern, claim):
            if claim not in pattern_match:
                pattern_match.append(claim)
    return pattern_match


#text_input = pn.widgets.TextInput(name="Claim search", placeholder="Enter claim here")
checkbox_group = pn.widgets.CheckBoxGroup(name="Checkbox Group", value=get_matches(pattern), options=get_matches(pattern))
#column = pn.Column(text_input, checkbox_group)
#column
checkbox_group

In [ ]:
claim_dict[pattern] = checkbox_group.value

filename = "claim_matching.json"

    
with open(filename, "r+") as jsonfile:
    dic = json.load(jsonfile)
    
    for key in claim_dict.keys():
        dic[key] = claim_dict[key]

with open(filename, "w") as jsonfile:
    json.dump(dic, jsonfile)


In [ ]:
df

In [ ]:
for claims in df["claims_proccesed"]:
    print(any((True for x in dic["vegan"] if x in claims)))
    

In [ ]:
#get_close_matches("recyclable", [claim for claim in claims_no_ingredients], 10, 0.7)

In [ ]:
#wnl = WordNetLemmatizer()
 
# Define function to lemmatize each word with its POS tag
 
# POS_TAGGER_FUNCTION : TYPE 1
#def pos_tagger(nltk_tag):
#    if nltk_tag.startswith('J'):
#        return wordnet.ADJ
#    elif nltk_tag.startswith('V'):
#        return wordnet.VERB
#    elif nltk_tag.startswith('N'):
#        return wordnet.NOUN
#    elif nltk_tag.startswith('R'):
#        return wordnet.ADV
#    else:         
#        return None
 

In [ ]:
#claims_lemmatized = []
#for sentence in claims_no_ingredients:
#    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
#    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
#    lemmatized_sentence = []
#    for word, tag in wordnet_tagged:
#        if tag is None:
#            # if there is no available tag, append the token as is
#            lemmatized_sentence.append(word)
#        else:       
#            # else use the tag to lemmatize the token
#            lemmatized_sentence.append(wnl.lemmatize(word, tag))
#    lemmatized_sentence = " ".join(lemmatized_sentence)
# 
#    claims_lemmatized.append(lemmatized_sentence)

In [ ]:
#Claim_ammount = Counter(claims_lemmatized)
#Claim_ammount.most_common()

In [ ]:
#get_close_matches("recyclable", [claim for claim in claims_no_ingredients], 10, 0.7)

To do:
    spelling checker to fix all the misspelled claims
    use difflib to find simmilar sentences?
    or https://stackoverflow.com/questions/63718559/finding-most-similar-sentences-among-all-in-python
    

In [ ]:
#short_claims = []
#for claim in claims_no_ingredients:
#    if len(claim) < 5:
#        short_claims.append(claim)

In [ ]:
#long_claims = []
#for claim in claims_no_ingredients:
#    if len(claim) > 30:
#        long_claims.append(claim)

first make manual list


#bert


NMF unsupervised clustering method


TF-IDF

Manual list:
vegetarian
green dot certified
recyclable
christmas
easter
valentine's day
king's day
gluten free
microwaveable
palm oil free
cocoa certified
nutriscore c
nutriscore d
nutriscore e
gmo free
fsc certified
vegan
lactose free
no preservatives
no artificial colors
no artificial flavors
certified organic
certified halal
fairtrade cocoa
free range eggs
milk free

manual_list = ["vegetarian",
"green dot certified",
"recyclable",
"christmas",
"easter",
"valentine's day",
"king's day",
"gluten free",
"microwaveable",
"palm oil free",
"cocoa certified",
"nutriscore c",
"nutriscore d",
"nutriscore e",
"gmo free",
"fsc certified",
"vegan",
"lactose free",
"no preservatives",
"no artificial colors",
"no artificial flavors",
"certified organic",
"certified halal",
"fairtrade cocoa",
"free range eggs",
"milk free"]

manual_list